# Comparison of iRhto1108_v2 and rhto-GEM

In [1]:
import pandas as pd
import numpy as np
#from tqdm import tqdm
import os
from copy import deepcopy
from os.path import join

import cobra
import re
import json
import openpyxl
#import gsm_custom_functions
import scipy.io as sio
from cobra.flux_analysis import flux_variability_analysis

In [2]:
dir_data = f"{os.path.expanduser('~/Downloads/')}"
dir_output = f"{os.path.expanduser('~/Desktop/')}"
model_cgh_filename = join(dir_data,"iRhtoC-gh.json")
model_chalmers = join(dir_data,"rhto.xml")

In [3]:
model_cgh = cobra.io.load_json_model(model_cgh_filename)
model_cgh.solver = 'cplex'

model_chal = cobra.io.read_sbml_model(model_chalmers)
model_chal.solver = 'cplex'


In [4]:
# iRhto1108 v2, Carbon limited, Hoang

with model_cgh as m:
    solution_updated = m.optimize()
    display(m.summary())
    fva_result_cgh = flux_variability_analysis(m, m.reactions)#,fraction_of_optimum=0.)
    

Metabolite,Reaction,Flux,C-Number,C-Flux
btn_c,EX_btn_c,3.902E-05,10,0.00%
ca2_e,EX_ca2_e,0.0005018,0,0.00%
cu2_e,EX_cu2_e,4.448E-05,0,0.00%
fe2_e,EX_fe2_e,0.0002981,0,0.00%
glc__D_e,EX_glc__D_e,5,6,100.00%
k_e,EX_k_e,0.2355,0,0.00%
mg2_e,EX_mg2_e,0.02482,0,0.00%
mn2_e,EX_mn2_e,3.668E-05,0,0.00%
nh4_e,EX_nh4_e,2.52,0,0.00%
o2_e,EX_o2_e,12.13,0,0.00%


In [5]:
# model from https://github.com/SysBioChalmers/rhto-GEM
with model_chal as m:
    medium = m.medium
    medium['r_1714'] = 5 # same glucose uptae basis as Hoang's model
    m.medium = medium

    solution_chal = m.optimize()
    display(m.summary())
    fva_result_chal = flux_variability_analysis(m, m.reactions)#,fraction_of_optimum=0.)
    

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654,1.662,0,0.00%
s_0565,r_1714,5,6,100.00%
s_0796,r_1832,0.08884,0,0.00%
s_0925,r_1861,4.044E-07,0,0.00%
s_1277,r_1992,11.98,0,0.00%
s_1324,r_2005,0.08884,0,0.00%
s_1468,r_2060,0.03438,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
s_0458,r_1672,-13.31,1,100.00%
s_0805,r_2100,-18.53,0,0.00%


In [6]:
# rhto-GEM, with lower glucose uptake rate from doi.org:10.1371/journal.pcbi.1011009
# Table S2, P9

with model_chal as m:
    medium = m.medium
    medium['r_1714'] = 2.096
    m.medium = medium
    solution_chal = m.optimize()
    display(m.summary())
    fva_result_chal = flux_variability_analysis(m, m.reactions)#,fraction_of_optimum=0.)
    

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654,0.6761,0,0.00%
s_0565,r_1714,2.096,6,100.00%
s_0796,r_1832,0.03614,0,0.00%
s_0925,r_1861,1.645E-07,0,0.00%
s_1277,r_1992,5.247,0,0.00%
s_1324,r_2005,0.03614,0,0.00%
s_1468,r_2060,0.01399,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
s_0458,r_1672,-5.785,1,100.00%
s_0805,r_2100,-7.909,0,0.00%


In [7]:
# what is the lowest glucose uptake that is able to match the observed growth rate?

with model_chal as m:
    medium = m.medium
    medium['r_1714'] = 1000 # allow temporarily large uptake
    m.medium = medium
    m.reactions.get_by_id('r_2111').lower_bound = 0.191 # set from data, Table S1 T9
    m.objective = {m.reactions.get_by_id('r_1714'): 1} # minimize glucose uptake
    solution_chal = m.optimize()
    display(m.summary())
    fva_result_chal = flux_variability_analysis(m, m.reactions)#,fraction_of_optimum=0.)
    

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654,0.7849,0,0.00%
s_0565,r_1714,2.417,6,100.00%
s_0796,r_1832,0.04196,0,0.00%
s_0925,r_1861,1.91E-07,0,0.00%
s_1277,r_1992,5.991,0,0.00%
s_1324,r_2005,0.04196,0,0.00%
s_1468,r_2060,0.01624,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
s_0458,r_1672,-6.615,1,100.00%
s_0805,r_2100,-9.081,0,0.00%


In [8]:
# Hoang v2, Carbon limited

# what is the lowest glucose uptake that is able to match the observed growth rate?

with model_cgh as m:
    medium = m.medium
    medium['EX_glc__D_e'] = 1000 # allow temporarily large uptake
    m.medium = medium
    m.reactions.get_by_id('BIOMASS').lower_bound = 0.191 # set from data
    m.objective = {m.reactions.get_by_id('EX_glc__D_e'): 1} # minimize glucose uptake
    solution_chal = m.optimize()
    display(m.summary())
    fva_result_chal = flux_variability_analysis(m, m.reactions)#,fraction_of_optimum=0.)
    

Metabolite,Reaction,Flux,C-Number,C-Flux
btn_c,EX_btn_c,1.91E-05,10,0.00%
ca2_e,EX_ca2_e,0.0002456,0,0.00%
cu2_e,EX_cu2_e,2.177E-05,0,0.00%
fe2_e,EX_fe2_e,0.0001459,0,0.00%
glc__D_e,EX_glc__D_e,2.462,6,100.00%
k_e,EX_k_e,0.1153,0,0.00%
mg2_e,EX_mg2_e,0.01215,0,0.00%
mn2_e,EX_mn2_e,1.795E-05,0,0.00%
nh4_e,EX_nh4_e,1.233,0,0.00%
o2_e,EX_o2_e,6.024,0,0.00%
